<a href="https://colab.research.google.com/github/Larai15/kubernetes/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
import requests         
from bs4 import BeautifulSoup  
import csv

headers = {
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'en-US,en;q=0.8',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Referer': 'http://www.wikipedia.org/',
        'Connection': 'keep-alive',
}

#AMAZON
def scrap_amazon(scrap_link):
  #scrap_link = "https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1"
  req = requests.get(scrap_link)
  soup = BeautifulSoup(req.content, 'html.parser')
  container = soup.findAll('div', attrs={'class':'a-section review aok-relative'})
  data = []
  for x in container:
      Reviewer = x.find('a', attrs={'class':'review-title'}).text
      ReviewerName = x.find('span', attrs={'class':'a-profile-name'}).text
      data.append([ReviewerName,Reviewer.strip("\n")])
  with open('amazon.csv', 'w', newline='') as csvfile:
      csvwriter = csv.writer(csvfile)
      csvwriter.writerow(['User','Review'])
      for review in data:
          csvwriter.writerow(review)

#JOKER

def scrape_joker(url):
    response = requests.get(url,headers=headers, timeout=10)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = []
    users  = []
    count  = 0
    for j in soup.findAll("a",attrs={'class':'title'}):
            if count >= 100:
                break
            reviews.append(j.text)
            count += 1
    count  = 0
    for j in soup.findAll("span",attrs={'class':'display-name-link'}):
            if count >= 100:
                break
            users.append(j.text)
            count += 1

    with open('imdb.csv', 'w') as csv_file:
        csvwriter = csv.writer(csv_file,delimiter=',')
        csvwriter.writerow(["USER","REVIEW"])
        for i in range(len(reviews)):
            csvwriter.writerow([users[i],reviews[i]])
def scrap_citeseer(scrap_link):
  req = requests.get(scrap_link)
  soup = BeautifulSoup(req.content, 'html.parser')
  container = soup.findAll('a', attrs={'class':'remove doc_details'})


  with open('citeseer.csv', 'w', newline='') as csvfile:
      csvwriter = csv.writer(csvfile)
      csvwriter.writerow(['PAPER'])
      for x in container[:100]:
          csvwriter.writerow([str(x.text).strip()])


if __name__=="__main__":
    url = "https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1"
    scrap_amazon(url)
    url = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"
    scrape_joker(url)
    url = "https://citeseerx.ist.psu.edu/search?q=natural+language+processing"
    scrap_citeseer(url)








# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [3]:
# Write your code here

from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word
from nltk.stem import PorterStemmer
import nltk
import re
import string
import csv
import pandas as pd 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop = stopwords.words('english')

def clean_text(train):
    result = {}
    split_train = train.split()

    #1.2
    train = train.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    train = train.translate(str.maketrans('', '', string.digits)) #remove digits

    train = " ".join(x for x in train.split() if x not in stop) #remove stop words

    train = train.lower() #lowercase

    st = PorterStemmer()
    train = " ".join(st.stem(x) for x in train.split())

    train = " ".join(Word(x).lemmatize() for x in train.split())
    return train
#Amazon
cleaned = []
with open("amazon.csv", encoding = "ISO-8859-1") as f:
  csvreader = list(csv.reader(f))
  csvreader[0].append("CLEANED DATA")
  cleaned.append(csvreader[0])
  for row in csvreader[1:]:
    train = row[1]
    train = clean_text(train)
    row.append(train)
    cleaned.append(row)

with open("amazon_clean.csv","w") as f:
    csvwriter = csv.writer(f)
    for l in cleaned:
       csvwriter.writerow(l)
  


#IMDB
cleaned = []
with open("imdb.csv", encoding = "ISO-8859-1") as f:
  csvreader = list(csv.reader(f))
  csvreader[0].append("CLEANED DATA")
  cleaned.append(csvreader[0])
  for row in csvreader[1:]:
    train = row[1]
    train = clean_text(train)
    row.append(train)
    cleaned.append(row)

with open("imdb_clean.csv","w") as f:
    csvwriter = csv.writer(f)
    for l in cleaned:
       csvwriter.writerow(l)

#CITESEER
cleaned = []
with open("citeseer.csv", encoding = "ISO-8859-1") as f:
  csvreader = list(csv.reader(f))
  csvreader[0].append("CLEANED DATA")
  cleaned.append(csvreader[0])
  for row in csvreader[1:]:
    train = row[0]
    train = clean_text(train)
    row.append(train)
    cleaned.append(row)

with open("citeseer_clean.csv","w") as f:
    csvwriter = csv.writer(f)
    for l in cleaned:
       csvwriter.writerow(l)





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [4]:
# Write your code here
import nltk
import csv
from pprint import pprint
import spacy
from spacy import displacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def train_pos_tag(train):
    tokens = nltk.word_tokenize(train)
    pos_tag = nltk.pos_tag(tokens)
    return pos_tag
def train_chunk(sentence):
    result = {}
    doc = nlp(sentence)
    # doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
    for ent in doc.ents: 
        if ent.label_ in result.keys():
            result[ent.label_] += 1
        else:
            result[ent.label_] = 1
    return result
     
def train_parts(post_tag):
    count = {"NN":0,"VB":0,"JJ":0,"RB":0}
    for tag in pos_tag:
        if tag[1] in count.keys():
            count[tag[1]] += 1
    return {"text":train,"noun":count["NN"],"verb":count["VB"],"adjective":count["JJ"],"adverb":count["RB"]}

#Amazon
print("\nAMAZON\n")
cleaned = []
with open("amazon_clean.csv", encoding = "ISO-8859-1") as f:
  csvreader = list(csv.reader(f))
  csvreader[0].append("CLEANED DATA")
  cleaned.append(csvreader[0])
  for row in csvreader[1:]:
    train = row[2]
    # print(train)
    pos_tag = train_pos_tag(train)
    parts = train_parts(pos_tag)
    print(parts)
    chunk = train_chunk(row[1])
    print(chunk)

#IMDB
print("\imdb\n")
cleaned = []
with open("imdb_clean.csv", encoding = "ISO-8859-1") as f:
  csvreader = list(csv.reader(f))
  csvreader[0].append("CLEANED DATA")
  cleaned.append(csvreader[0])
  for row in csvreader[1:]:
    train = row[2]
    pos_tag = train_pos_tag(train)
    parts = train_parts(pos_tag)
    print(parts)
    chunk = train_chunk(row[1])
    print(chunk)



#CITESEER
print("\nCITESEER\n")
cleaned = []
with open("citeseer_clean.csv", encoding = "ISO-8859-1") as f:
  csvreader = list(csv.reader(f))
  csvreader[0].append("CLEANED DATA")
  cleaned.append(csvreader[0])
  for row in csvreader[1:]:
    train = row[1]
    pos_tag = train_pos_tag(train)
    parts = train_parts(pos_tag)
    print(parts)
    chunk = train_chunk(row[1])
    print(chunk)






[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.

AMAZON

\imdb

{'text': 'a viewer actual went tiff wit film didnt want believ hype absolut masterpiec phoenix certifi legend', 'noun': 11, 'verb': 0, 'adjective': 2, 'adverb': 0}
{'ORG': 1, 'NORP': 1, 'GPE': 1}
{'text': 'outstand movi haunt perform best charact develop ever seen', 'noun': 5, 'verb': 1, 'adjective': 0, 'adverb': 1}
{}
{'text': 'onli certain peopl relat', 'noun': 3, 'verb': 0, 'adjective': 1, 'adverb': 0}
{}
{'text': 'must put smile satisfact heath ledger face right heaven', 'noun': 4, 'verb': 1, 'adjective': 1, 'adverb': 2}
{}
{'text': 'the hype real', 'noun': 1, 'verb

 A constituency parse breaks text into sub-phrases in such a way that there a terminals which are the words in the sentences and unlabeled edges while dependency parsing is the connection of relationships between words.
